## Building the CC App 
We're building a modern call center application that leverages real-time voice processing and AI capabilities to handle customer interactions.

### Building an Insurance Coverage Voice Assistant
This workshop will guide you through building a voice-enabled insurance policy assistant that helps customers understand their healthcare coverage. We'll take an existing voicebot system and enhance it to become a specialized insurance coverage checker, capable of accessing policy documents and providing accurate coverage information in real-time.

Workshop Objectives {#objectives}
By the end of this workshop, you will have built an AI-powered system that can:

- Interact naturally with customers using voice
- Access and interpret insurance policy documents
- Provide accurate coverage information for specific medical services
- Maintain a professional and helpful conversation flow
- Handle common customer inquiries about healthcare coverage

**Prerequisites**
Before starting this workshop, ensure you have:
The provided voice interaction system (p1uc1_realtime_api_converse_step2.py) from earlier steps 


Each customer's policy is stored in a separate text file using a standardized format:
```
policy_documents/
    └── insurance_policy_[customer_id].txt
`
First, let's create our policy documents in a specific format. \
Create a folder named policy_documents and save the following files: \
File: policy_documents/insurance_policy_12345.txt:

```
Customer Number: 12345
Name: John Smith
Policy Type: Premium Health Insurance Plan - AtlasMedical Insurance

Coverage Details for Specialist Visits:

Cardiologist Visits:
- In-network copay: $45
- Coverage: 85% after deductible
- Pre-authorization required for non-emergency procedures
- Referral from primary care physician required

Endocrinologist Visits:
- In-network copay: $40
- Coverage: 90% after deductible
- No pre-authorization required for standard visits
- Annual diabetes check-ups fully covered

General Coverage Terms:
- Annual deductible: $1,500
- Out-of-pocket maximum: $5,000
- In-network coverage: 80-90%
- Out-of-network coverage: 50-60%
```



## STEP1 - Modify the realtime API Converse Agent to adopt to the new Insurance Coverage Voice Assistant role 

#### Modified System Instructions:

We will change the basic AI assistant prompt to a specific insurance agent persona so that the instructions now focus on insurance-related interactions and procedures. \
We also need to add specific guidance about collecting customer IDs to match and providing policy information.

The system now starts with a proper insurance-specific greeting
It follows a logical flow: greeting → request ID → get query → provide information
Each state has appropriate responses and next steps

#### Added Insurance-Specific Components:

- We need to integrate a class e.g.AtlasMedicalAgent class which handles insurance policy lookups and interpretation. \
- Add conversation state tracking to remember customer IDs and context.
- Included conversation context storage to maintain coherent interactions.
- The AI now has detailed instructions about being an insurance agent
- It knows to ask for customer ID before providing any policy information
- The responses are tailored to insurance queries


#### Enhanced Response Handling:

Modified the text processing to route through our insurance agent.
Added handling for insurance-specific queries and responses.
Included context updating to maintain conversation flow.


#### State Management:

Added tracking for current customer ID.
Included conversation context storage for maintaining interaction history.
Added support for multi-turn conversations about insurance coverage.

We added a dedicated InsuranceConversationState class
This tracks where we are in the conversation
It ensures we get necessary information before providing policy details


#### Enhanced Feedback:

The system provides clear status messages about what's happening
It confirms when it hears customer ID numbers
It guides customers through the process step by step

The core audio processing and websocket handling remain unchanged because they're working well. We're primarily adding the insurance-specific logic layer on top of the existing infrastructure.
These modifications transform the general-purpose conversation system into a specialized insurance assistant while maintaining all the sophisticated voice processing capabilities of the original system.







State Management:





In [ ]:
# Modified ConversationSystem class
class ConversationSystem:
    def __init__(self):
        load_dotenv()
        self.api_key = os.getenv("AZURE_OPENAI_API_KEY")
        if not self.api_key:
            raise ValueError("AZURE_OPENAI_API_KEY not found")
            
        # Initialize our insurance-specific components
        self.insurance_agent = AtlasMedicalAgent()
        
        self.url = (
            "wss://aoai-ep-swedencentral02.openai.azure.com/openai/realtime?"
            f"api-version=2024-10-01-preview&deployment=gpt-4o-realtime-preview&"
            f"api-key={self.api_key}"
        )
        
        self.audio_processor = AudioProcessor()
        self.streams = {'input': None, 'output': None}
        
        # Add conversation state tracking
        self.current_customer_id = None
        self.conversation_context = {}

    async def setup_websocket_session(self, websocket):
        # Modify the session configuration to be insurance-specific
        session_config = {
            "type": "session.update",
            "session": {
                "voice": "alloy",
                # Change instructions to define insurance agent persona
                "instructions": """You are Alex, a friendly insurance coverage specialist at 
                    AtlasMedical Insurance. Maintain a professional yet warm tone. Your role is to:
                    1. Help customers understand their insurance coverage
                    2. Ask for their customer ID when coverage questions arise
                    3. Provide clear, accurate policy information
                    4. Keep responses concise and focused on insurance matters
                    Begin by greeting the customer and asking how you can assist them today.""",
                "modalities": ["audio", "text"],
                "input_audio_format": "pcm16",
                "output_audio_format": "pcm16",
                "turn_detection": {
                    "type": "server_vad",
                    "threshold": 0.3,
                    "prefix_padding_ms": 150,
                    "silence_duration_ms": 600
                }
            }
        }
        
        await websocket.send(json.dumps(session_config))
        
        while True:
            response = json.loads(await websocket.recv())
            if response["type"] == "session.created":
                break
            if response["type"] == "error":
                raise Exception(f"Session setup failed: {response}")

    async def handle_response(self, websocket):
        """Enhanced response handling for insurance queries"""
        self.audio_processor.is_speaking = True
        
        try:
            # Track the complete text response for this turn
            current_response_text = ""
            
            while True:
                response = json.loads(await websocket.recv())
                
                if response["type"] == "response.text":
                    # Process customer's speech through insurance agent
                    customer_text = response.get("text", "")
                    agent_response = await self.insurance_agent.process_customer_input(customer_text)
                    
                    # Send the agent's response back
                    await websocket.send(json.dumps({
                        "type": "text.generate",
                        "text": agent_response
                    }))
                
                elif response["type"] == "response.audio.delta":
                    if "delta" in response:
                        try:
                            audio_data = response["delta"].strip()
                            padding = -len(audio_data) % 4
                            if padding:
                                audio_data += "=" * padding
                            
                            audio = np.frombuffer(
                                base64.b64decode(audio_data), 
                                dtype=np.int16
                            )
                            self.streams['output'].write(audio)
                            
                        except Exception as e:
                            print(f"Audio processing error: {e}")
                            
                elif response["type"] == "response.done":
                    # Update conversation context if needed
                    if self.current_customer_id:
                        self.conversation_context[self.current_customer_id] = {
                            'last_query': current_response_text,
                            'timestamp': datetime.now()
                        }
                    break
                    
        finally:
            self.audio_processor.is_speaking = False

    async def run(self):
        # The run method remains largely the same
        await self.setup_audio()
        
        async with websockets.connect(self.url) as ws:
            await self.setup_websocket_session(ws)
            print("AtlasMedical Insurance Assistant is ready")
            
            while True:
                if self.audio_processor.should_process():
                    audio_data = self.audio_processor.reset()
                    await self.send_audio(ws, audio_data)
                    await self.handle_response(ws)
                await asyncio.sleep(0.05)